### Model Candidate - Decision Tree

In [9]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [3]:
from sklearn import tree, preprocessing
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, cross_validate
from sklearn.metrics import recall_score

In [12]:
os.listdir('../data/pickle_files/')

['df_pickle_w_all_stats', 'df_pickle', 'df_pickle_w_time_stats']

In [26]:
#Loading df1 after it has been augmented in iteration 2:
df = pd.read_pickle('../data/pickle_files/df_pickle_w_time_stats')
#Excluding the duration variable as it cannot be used in our baseline
df = df.drop(columns = ['duration'])

In [27]:
#Checking dtypes have loaded correctly (should return empty index)
df.select_dtypes(exclude = ['float64', 'int64']).columns


df.drop(columns=['Date'], inplace=True)

In [32]:
def scale_noncat_only(df):
    X_temp_noncat = df.select_dtypes(exclude=['int8'])
    X_temp_cat = df.select_dtypes(include=['int8'])
    scaler = preprocessing.StandardScaler().fit(X_temp_noncat)
    X_transformed = scaler.transform(X_temp_noncat)
    X = pd.concat([pd.DataFrame(X_transformed, columns =X_temp_noncat.columns), X_temp_cat], axis = 1)
    X = X.drop(columns=['y'])
    y = df['y']
    #will work with numpy arrays
    y = np.array(y)
    X = np.array(X)
    
    return X, y

In [33]:
X, y = scale_noncat_only(df)

In [34]:
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, cross_validate

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 4)

In [37]:
#class sklearn.tree.DecisionTreeClassifier(criterion=’gini’, splitter=’best’, max_depth=None, 
#                                          min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
#                                          max_features=None, random_state=None, max_leaf_nodes=None, 
#                                          min_impurity_decrease=0.0, min_impurity_split=None, class_weight=None, 
#                                          presort=False)

#try class_weight = balanced

clf = tree.DecisionTreeClassifier(criterion='gini',
                             max_depth=10,
                             min_samples_leaf = 1000,
                             max_leaf_nodes = 10)

In [38]:
clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
            max_features=None, max_leaf_nodes=10,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1000, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [40]:
scoring = ['recall', 'precision']
for score in scoring:
    print('%score Cross Validated Scores: ' %score)
    print(cross_val_score(clf, X_train, y_train, cv = 10, scoring=score))
    print('\n')

recallcore Cross Validated Scores: 
[0.27873563 0.28448276 0.31034483 0.28448276 0.29682997 0.29971182
 0.28530259 0.23342939 0.26512968 0.2795389 ]


precisioncore Cross Validated Scores: 
[0.5        0.53804348 0.56842105 0.495      0.55080214 0.51485149
 0.48768473 0.49693252 0.46938776 0.53296703]




In [50]:
#Use cross_validate rather than cross val score in order to compare training and val scores

#### 